In [29]:
# import itertools
import os
import sys
from pathlib import Path

import cv2
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import skimage.io

from collections import defaultdict
from tqdm.auto import tqdm
from joblib import Parallel, delayed
import re
import h5py
import napari
import tifffile as tiff
import seaborn as sns
import pickle

In [30]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [31]:
p_dir = (Path().cwd().parents[0]).absolute()

module_path = str(p_dir / "src")
 
if module_path not in sys.path:
    sys.path.append(module_path)

In [32]:
data_dir = (Path().cwd().parents[0] / 'data').absolute()

df_meta_path = data_dir / 'sensitivity_v2' / 'metadata' / 'imgs_reg.csv'
df_imgs = pd.read_csv(df_meta_path)


In [33]:
df_imgs

,Condition,FOV,Path
0,100nM,C,Y:\coskun-lab\Thomas\23_PLA_revision\data\sens...
1,10nM,E,Y:\coskun-lab\Thomas\23_PLA_revision\data\sens...
2,25nM,D,Y:\coskun-lab\Thomas\23_PLA_revision\data\sens...
3,5nM,F,Y:\coskun-lab\Thomas\23_PLA_revision\data\sens...
4,Control,B,Y:\coskun-lab\Thomas\23_PLA_revision\data\sens...


# Plot examples

Revision Figure on tratement 
Need to look at FOV 6 and 7 and compared control and treated

In [34]:
import napari 

# remove first and last consecutive zeros along dimension
def removeFirstAndLast(zeroArray):
    
    # remove first
    for ii, val in enumerate(zeroArray):
        if np.sum(zeroArray[:ii + 1]) != ii + 1: # all consecutive trues
            break # exit loop
            
        else:
            lowest = ii # index of last of the first consecutive zeros
            
    # remove last
    for ii, val in enumerate(np.flip(zeroArray)):
        if np.sum(zeroArray[:ii + 1]) != ii + 1: # all consecutive trues
            break # exit loop
            
        else:
            highest = len(zeroArray) - ii # index of first of the last consecutive zeros            
    
    return (lowest, highest)

# remove black borders from napari screenshot (2D image)
def removeBorders(RGB):
    rowMin = []
    rowMax = []
    colMin = []
    colMax = []
    for jj in range(3): # each RGB
        
        img = RGB[:, :, jj]
        # find first and last rows and cols of all zeros
        # remove any rows of zeros
        zeroRows = np.all(img == 0, axis = 1)
        zeroCols = np.all(img == 0, axis = 0)

        posRows = removeFirstAndLast(zeroRows)
        posCols = removeFirstAndLast(zeroCols)            
        
        rowMin.append(posRows[0])
        rowMax.append(posRows[1])
        colMin.append(posCols[0])
        colMax.append(posCols[1])
        
    # make sure border signal is not cut out
    rowMin = np.min(rowMin)
    rowMax = np.max(rowMax)
    colMin = np.min(colMin)
    colMax = np.max(colMax)
        
    cropped = RGB[rowMin: rowMax, colMin: colMax, :] 
    
    return cropped

def get_loc_ppi(df, Condition, FOV, x_min, y_min, x_max, y_max, PPI_name):
    df_ppi = df[(df.Condition == Condition) & (df.FOV == FOV)]
    df_ppi = df_ppi[df_ppi.PPI == PPI_name]
    df_ppi = df_ppi[(df_ppi.column > x_min) & (df_ppi.column < x_max) 
                    & (df_ppi.row > y_min) & (df_ppi.row < y_max)] 
    points = df_ppi[['row', 'column']].to_numpy()
    points = points - np.array([y_min, x_min])
    return points

In [35]:
# Read PPI 
PPI_save_path =  data_dir / 'sensitivity_v2'/ 'PPI'

dfs = []
for path in os.listdir(PPI_save_path):
    if 'csv' in path:
        df = pd.read_csv(PPI_save_path / path)
        dfs.append(df)

df = pd.concat(dfs)
df = df.rename(columns={'x': 'row', 'y': 'column'})
df.head()

,Cyto,row,column,PPI,Nuclei,Condition,FOV
0,0,1826,503,Sox2/Oct4,0,100nM,C
1,72,1345,374,Sox2/Oct4,0,100nM,C
2,0,951,2665,Sox2/Oct4,0,100nM,C
3,0,2741,60,Sox2/Oct4,0,100nM,C
4,0,203,1450,Sox2/Oct4,0,100nM,C


In [36]:
image_dir = (Path().cwd().parents[0] / 'figures').absolute() / 'PLA_scatter'
image_dir.mkdir(parents=True, exist_ok=True)

## B05

In [37]:
# Get images
index = 4
row = df_imgs.iloc[index]
print(row)
path = row.Path

with h5py.File(path, 'r') as f:
    imgs = f['imgs'][:]
    markers = f['imgs'].attrs['Marker']


Condition                                              Control
FOV                                                          B
Path         Y:\coskun-lab\Thomas\23_PLA_revision\data\sens...
Name: 4, dtype: object


In [38]:
# Define matching bbox to figures
x_min, y_min = 1292, 712
x_max, y_max = 1720, 1204
DAPI_ch = 0

# Get DAPI image
dapiImg = imgs[0, y_min:y_max, x_min:x_max]

# Get ppi location
PPI_names = ['Sox2/Oct4', 'p-P90rsk/NF-Kb', 'Cyclin D1/CDK4', 'Bim/Tom20']
points = [get_loc_ppi(df, row.Condition, row.FOV, x_min, y_min, x_max, y_max, PPI_name) for PPI_name in PPI_names]


In [39]:
# Napari viewer
viewer = napari.view_image(dapiImg, colormap='blue')
points_layer = viewer.add_points(points[0], size=8,face_color='magenta',)
points_layer = viewer.add_points(points[1], size=8,face_color='lime',)

# Get screenshot
screenshot = viewer.screenshot(size=np.array(dapiImg.shape)*2)
viewer.close()
# Save image
tiff.imsave(image_dir / f'sensitivity_B05_1.tif', removeBorders(screenshot))

C:\Users\thu71\AppData\Local\Temp\ipykernel_7740\1515763897.py:10: DeprecationWarning: <tifffile.imsave> is deprecated. Use tifffile.imwrite
  tiff.imsave(image_dir / f'sensitivity_B05_1.tif', removeBorders(screenshot))


In [40]:
# Napari viewer
viewer = napari.view_image(dapiImg, colormap='blue')
points_layer = viewer.add_points(points[2], size=8,face_color='magenta',)
points_layer = viewer.add_points(points[3], size=8,face_color='lime',)

# Get screenshot
screenshot = viewer.screenshot(size=np.array(dapiImg.shape)*2)
viewer.close()
# Save image
tiff.imsave(image_dir / f'sensitivity_B05_2.tif', removeBorders(screenshot))

C:\Users\thu71\AppData\Local\Temp\ipykernel_7740\2455099806.py:10: DeprecationWarning: <tifffile.imsave> is deprecated. Use tifffile.imwrite
  tiff.imsave(image_dir / f'sensitivity_B05_2.tif', removeBorders(screenshot))


## C05

In [41]:
# Get images
index = 0
row = df_imgs.iloc[index]
print(row)
path = row.Path

with h5py.File(path, 'r') as f:
    imgs = f['imgs'][:]
    markers = f['imgs'].attrs['Marker']


Condition                                                100nM
FOV                                                          C
Path         Y:\coskun-lab\Thomas\23_PLA_revision\data\sens...
Name: 0, dtype: object


In [42]:
# Define matching bbox to figures
x_min, y_min = 1464,2408
x_max, y_max = 1852,2824
DAPI_ch = 0

# Get DAPI image
dapiImg = imgs[0, y_min:y_max, x_min:x_max]

# Get ppi location
PPI_names = ['Sox2/Oct4', 'p-P90rsk/NF-Kb', 'Cyclin D1/CDK4', 'Bim/Tom20']
points = [get_loc_ppi(df, row.Condition, row.FOV, x_min, y_min, x_max, y_max, PPI_name) for PPI_name in PPI_names]


In [43]:
# Napari viewer
viewer = napari.view_image(dapiImg, colormap='blue')
points_layer = viewer.add_points(points[0], size=8,face_color='magenta',)
points_layer = viewer.add_points(points[1], size=8,face_color='lime',)

# Get screenshot
screenshot = viewer.screenshot(size=np.array(dapiImg.shape)*2)
viewer.close()
# Save image
tiff.imsave(image_dir / f'sensitivity_C05_1.tif', removeBorders(screenshot))

C:\Users\thu71\AppData\Local\Temp\ipykernel_7740\1486595845.py:10: DeprecationWarning: <tifffile.imsave> is deprecated. Use tifffile.imwrite
  tiff.imsave(image_dir / f'sensitivity_C05_1.tif', removeBorders(screenshot))


In [44]:
# Napari viewer
viewer = napari.view_image(dapiImg, colormap='blue')
points_layer = viewer.add_points(points[2], size=8,face_color='magenta',)
points_layer = viewer.add_points(points[3], size=8,face_color='lime',)

# Get screenshot
screenshot = viewer.screenshot(size=np.array(dapiImg.shape)*2)
viewer.close()
# Save image
tiff.imsave(image_dir / f'sensitivity_C05_2.tif', removeBorders(screenshot))

C:\Users\thu71\AppData\Local\Temp\ipykernel_7740\3705326910.py:10: DeprecationWarning: <tifffile.imsave> is deprecated. Use tifffile.imwrite
  tiff.imsave(image_dir / f'sensitivity_C05_2.tif', removeBorders(screenshot))


## D05

In [45]:
# Get images
index = 2
row = df_imgs.iloc[index]
print(row)
path = row.Path

with h5py.File(path, 'r') as f:
    imgs = f['imgs'][:]
    markers = f['imgs'].attrs['Marker']


Condition                                                 25nM
FOV                                                          D
Path         Y:\coskun-lab\Thomas\23_PLA_revision\data\sens...
Name: 2, dtype: object


In [46]:
# Define matching bbox to figures
x_min, y_min = 400, 1580
x_max, y_max = 740, 1892

# Get DAPI image
dapiImg = imgs[0, y_min:y_max, x_min:x_max]

# Get ppi location
PPI_names = ['Sox2/Oct4', 'p-P90rsk/NF-Kb', 'Cyclin D1/CDK4', 'Bim/Tom20']
points = [get_loc_ppi(df, row.Condition, row.FOV, x_min, y_min, x_max, y_max, PPI_name) for PPI_name in PPI_names]


In [47]:
# Napari viewer
viewer = napari.view_image(dapiImg, colormap='blue')
points_layer = viewer.add_points(points[0], size=8,face_color='magenta',)
points_layer = viewer.add_points(points[1], size=8,face_color='lime',)

# Get screenshot
screenshot = viewer.screenshot(size=np.array(dapiImg.shape)*2)
viewer.close()
# Save image
tiff.imsave(image_dir / f'sensitivity_D05_1.tif', removeBorders(screenshot))

C:\Users\thu71\AppData\Local\Temp\ipykernel_7740\648470091.py:10: DeprecationWarning: <tifffile.imsave> is deprecated. Use tifffile.imwrite
  tiff.imsave(image_dir / f'sensitivity_D05_1.tif', removeBorders(screenshot))


In [48]:
# Napari viewer
viewer = napari.view_image(dapiImg, colormap='blue')
points_layer = viewer.add_points(points[2], size=8,face_color='magenta',)
points_layer = viewer.add_points(points[3], size=8,face_color='lime',)

# Get screenshot
screenshot = viewer.screenshot(size=np.array(dapiImg.shape)*2)
viewer.close()
# Save image
tiff.imsave(image_dir / f'sensitivity_D05_2.tif', removeBorders(screenshot))

C:\Users\thu71\AppData\Local\Temp\ipykernel_7740\4243784634.py:10: DeprecationWarning: <tifffile.imsave> is deprecated. Use tifffile.imwrite
  tiff.imsave(image_dir / f'sensitivity_D05_2.tif', removeBorders(screenshot))


## E05

In [49]:
# Get images
index = 1
row = df_imgs.iloc[index]
print(row)
path = row.Path

with h5py.File(path, 'r') as f:
    imgs = f['imgs'][:]
    markers = f['imgs'].attrs['Marker']


Condition                                                 10nM
FOV                                                          E
Path         Y:\coskun-lab\Thomas\23_PLA_revision\data\sens...
Name: 1, dtype: object


In [50]:
# Define matching bbox to figures
x_min, y_min = 2198, 2712
x_max, y_max = 2546, 3012
DAPI_ch = 0

# Get DAPI image
dapiImg = imgs[0, y_min:y_max, x_min:x_max]

# Get ppi location
PPI_names = ['Sox2/Oct4', 'p-P90rsk/NF-Kb', 'Cyclin D1/CDK4', 'Bim/Tom20']
points = [get_loc_ppi(df, row.Condition, row.FOV, x_min, y_min, x_max, y_max, PPI_name) for PPI_name in PPI_names]


In [51]:
# Napari viewer
viewer = napari.view_image(dapiImg, colormap='blue')
points_layer = viewer.add_points(points[0], size=8,face_color='magenta',)
points_layer = viewer.add_points(points[1], size=8,face_color='lime',)

# Get screenshot
screenshot = viewer.screenshot(size=np.array(dapiImg.shape)*2)
viewer.close()
# Save image
tiff.imsave(image_dir / f'sensitivity_E05_1.tif', removeBorders(screenshot))

C:\Users\thu71\AppData\Local\Temp\ipykernel_7740\807967018.py:10: DeprecationWarning: <tifffile.imsave> is deprecated. Use tifffile.imwrite
  tiff.imsave(image_dir / f'sensitivity_E05_1.tif', removeBorders(screenshot))


In [52]:
# Napari viewer
viewer = napari.view_image(dapiImg, colormap='blue')
points_layer = viewer.add_points(points[2], size=8,face_color='magenta',)
points_layer = viewer.add_points(points[3], size=8,face_color='lime',)

# Get screenshot
screenshot = viewer.screenshot(size=np.array(dapiImg.shape)*2)
viewer.close()
# Save image
tiff.imsave(image_dir / f'sensitivity_E05_2.tif', removeBorders(screenshot))

C:\Users\thu71\AppData\Local\Temp\ipykernel_7740\2462245639.py:10: DeprecationWarning: <tifffile.imsave> is deprecated. Use tifffile.imwrite
  tiff.imsave(image_dir / f'sensitivity_E05_2.tif', removeBorders(screenshot))


## F05

In [53]:
# Get images
index = 3
row = df_imgs.iloc[index]
print(row)
path = row.Path

with h5py.File(path, 'r') as f:
    imgs = f['imgs'][:]
    markers = f['imgs'].attrs['Marker']


Condition                                                  5nM
FOV                                                          F
Path         Y:\coskun-lab\Thomas\23_PLA_revision\data\sens...
Name: 3, dtype: object


In [54]:
# Define matching bbox to figures
x_min, y_min = 772, 584
x_max, y_max = 1148, 960
DAPI_ch = 0

# Get DAPI image
dapiImg = imgs[0, y_min:y_max, x_min:x_max]

# Get ppi location
PPI_names = ['Sox2/Oct4', 'p-P90rsk/NF-Kb', 'Cyclin D1/CDK4', 'Bim/Tom20']
points = [get_loc_ppi(df, row.Condition, row.FOV, x_min, y_min, x_max, y_max, PPI_name) for PPI_name in PPI_names]


In [55]:
# Napari viewer
viewer = napari.view_image(dapiImg, colormap='blue')
points_layer = viewer.add_points(points[0], size=8,face_color='magenta',)
points_layer = viewer.add_points(points[1], size=8,face_color='lime',)

# Get screenshot
screenshot = viewer.screenshot(size=np.array(dapiImg.shape)*2)
viewer.close()
# Save image
tiff.imsave(image_dir / f'sensitivity_F05_1.tif', removeBorders(screenshot))

C:\Users\thu71\AppData\Local\Temp\ipykernel_7740\4032783075.py:10: DeprecationWarning: <tifffile.imsave> is deprecated. Use tifffile.imwrite
  tiff.imsave(image_dir / f'sensitivity_F05_1.tif', removeBorders(screenshot))


In [56]:
# Napari viewer
viewer = napari.view_image(dapiImg, colormap='blue')
points_layer = viewer.add_points(points[2], size=8,face_color='magenta',)
points_layer = viewer.add_points(points[3], size=8,face_color='lime',)

# Get screenshot
screenshot = viewer.screenshot(size=np.array(dapiImg.shape)*2)
viewer.close()
# Save image
tiff.imsave(image_dir / f'sensitivity_F05_2.tif', removeBorders(screenshot))

C:\Users\thu71\AppData\Local\Temp\ipykernel_7740\2081226841.py:10: DeprecationWarning: <tifffile.imsave> is deprecated. Use tifffile.imwrite
  tiff.imsave(image_dir / f'sensitivity_F05_2.tif', removeBorders(screenshot))
